In [43]:
with open('./training_data/Roman-Urdu.txt', 'r', encoding='utf-8') as f:
    romanUrduLines = f.readlines()[:200]

with open('./training_data/Urdu.txt', 'r', encoding='utf-8') as f:
    urduLines = f.readlines()[:200]


    

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random


In [45]:
with open("./training_data/Roman-Urdu.txt", "r", encoding="utf-8") as f:
    roman_urdu_data = f.read()

with open("./training_data/Urdu.txt", "r", encoding="utf-8") as f:
    urdu_data = f.read()



In [46]:
print(urdu_data.splitlines())

['وکیل ', 'وکیل    ', 'وکیل     ایک ایسی شخصیت کو کہا جاتا ہے کہ جو دوسرے  اپنے صارف  کی جانب سے یا اسکی بابت گفتگو کرے   اس مضمون میں یہ گفتگو قانون سے متعلق تصور کی گئی ہے اور اس وجہ سے یہ مضمون صرف قانونی وکلا  کے بارے میں ذکر کرتا ہے', 'وکیل  قانون ', 'وکیل  قانون  ایک شخص جسے دوسرے شخص کی جگہ کام کرنے یا     کی نمائندگی کرنے کا اختیار حاصل ہوتا ہے  وکیل  سفر  ایک شخص جو تعطیلات اور سفر کا بندوبست کرتا ہے  خفیہ وکیل  ایک جاسوس ', 'وکیل بازار ضلع   لاطینی       ترکمانستان کا ایک ترکمانستان کے اضلاع جو صوبہ ماری میں واقع ہے', 'وکیل والا ریلوے اسٹیشن وکیل والا میں واقع ہے', 'وکیل والا ریلوے اسٹیشن پاکستان میں واقع ہے', 'وکیپیڈیا', 'وکیپیڈیا انگریزی پر یہ تصویر ميں نے ہی اپلوڈ کی تھی', 'وکیپیڈیا خود کسی بھی ترتیب کو نافذ نہیں کرتا', 'وکیپیڈیا لوگو کے لیے نستعلیق خطاطی', 'وکیپیڈیا میں اس سطر سے آپ تصاویر سادہ طریقے سے شامل کرسکتے ہیں ', 'وکیپیڈیا میں خوش آمدید', 'وکیپیڈیا میں کسی کے کو اس بات کی وجہ سے فوقیت حاصل نہیں کہ اس نے کس قدر وکیپیڈیا میں اضافہ کیا ہے', 'وکیپیڈیا پالیسی ', 'وکیپ

In [47]:
roman_urdu_lines = roman_urdu_data.splitlines()
urdu_lines = urdu_data.splitlines()

Output of urdu text has empty spaces which we need to remove

In [48]:
urdu_lines[2].split(' ')[:10]

['وکیل', '', '', '', '', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا']

In [49]:
print(urdu_lines[2].replace('  ',' '))

وکیل   ایک ایسی شخصیت کو کہا جاتا ہے کہ جو دوسرے اپنے صارف کی جانب سے یا اسکی بابت گفتگو کرے  اس مضمون میں یہ گفتگو قانون سے متعلق تصور کی گئی ہے اور اس وجہ سے یہ مضمون صرف قانونی وکلا کے بارے میں ذکر کرتا ہے


In [50]:
[word for word in urdu_lines[2].split(' ') if word != ''][:10]

['وکیل', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا', 'جاتا', 'ہے', 'کہ', 'جو']

In [90]:
for i, line in enumerate(urdu_lines):
    # Remove empty spaces for each line and also truncate to 10 words
    if line != '':
        urdu_lines[i] = ' '.join([word for word in line.split(' ') if word != ''][:10])

urdu_lines = [line.strip() for line in urdu_lines if line.strip()]

We can see that now empty spaces are gone and the sentences are truncated to 10 words

In [91]:
urdu_lines[2].split(' ')
    

['وکیل', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا', 'جاتا', 'ہے', 'کہ', 'جو']

Now we do the same for roman urdu

In [92]:
for i, line in enumerate(roman_urdu_lines):
    # Remove empty spaces for each line and also truncate to 10 words
    roman_urdu_lines[i] = ' '.join([word for word in line.split(' ') if word != ''][:10])

In [93]:
roman_urdu_lines[2].split(' ')

['wakeel',
 'aik',
 'aisi',
 'shakhsiyat',
 'ko',
 'kaha',
 'jata',
 'hai',
 'ke',
 'jo']

In [94]:
print(roman_urdu_lines, urdu_lines[3].split())

['wakeel', 'wakeel', 'wakeel aik aisi shakhsiyat ko kaha jata hai ke jo', 'wakeel qanoon', 'wakeel qanoon 1 shakhs jisay dosray shakhs ki jagah kaam', 'wakeel bazaar zila lateeni turkmenistan ka aik turkmenistan ke azlaa', 'wakeel wala railway station wakeel wala mein waqay hai', 'wakeel wala railway station Pakistan mein waqay hai', 'wikipedia', 'wikipedia angrezi par yeh tasweer mein ne hi aplod ki', 'wikipedia khud kisi bhi tarteeb ko nafiz nahi karta', 'wikipedia logo ke liye nastalik khtati', 'wikipedia mein is satar se aap tasaveer saada tareeqay se', 'wikipedia mein khush aamdeed', 'wikipedia mein kisi ke ko is baat ki wajah se', 'wikipedia policy', 'wikipedia par sirf hazaar mazmoon hain jin mein se asal', 'wikipedia par har Sarif ka abna aik safha hota hai', 'wikipedia ke adad o shumaar', 'wikipedia ke sarfeen ki asal tasaveer', 'wikipedia ke kisi bhi safha ke dayen janib talaash ka', 'wagar nah dil to usay rokna hi chahta hai', 'vgn shehar zila larkana Sindh ka aik khobsorat 

In [95]:
def getUniqueWords(lines):

    vocab = []
    for i in lines:
        toks = i.split()

        for j in toks:
            if j not in vocab:
                vocab.append(j)

    return vocab

Get Vocab of roman urdu and urdu

In [96]:
import torchtext.vocab as vocab

vocab_roman = getUniqueWords(roman_urdu_lines)
vocab_urdu = getUniqueWords(urdu_lines)

Urdu Vocab only first 35 words

In [97]:
print(len(vocab_urdu))
vocab_urdu[:35]

5223


['وکیل',
 'ایک',
 'ایسی',
 'شخصیت',
 'کو',
 'کہا',
 'جاتا',
 'ہے',
 'کہ',
 'جو',
 'قانون',
 'شخص',
 'جسے',
 'دوسرے',
 'کی',
 'جگہ',
 'کام',
 'بازار',
 'ضلع',
 'لاطینی',
 'ترکمانستان',
 'کا',
 'کے',
 'اضلاع',
 'والا',
 'ریلوے',
 'اسٹیشن',
 'میں',
 'واقع',
 'پاکستان',
 'وکیپیڈیا',
 'انگریزی',
 'پر',
 'یہ',
 'تصویر']

In [98]:
print(len(vocab_roman))
vocab_roman[:35]

5140


['wakeel',
 'aik',
 'aisi',
 'shakhsiyat',
 'ko',
 'kaha',
 'jata',
 'hai',
 'ke',
 'jo',
 'qanoon',
 '1',
 'shakhs',
 'jisay',
 'dosray',
 'ki',
 'jagah',
 'kaam',
 'bazaar',
 'zila',
 'lateeni',
 'turkmenistan',
 'ka',
 'azlaa',
 'wala',
 'railway',
 'station',
 'mein',
 'waqay',
 'Pakistan',
 'wikipedia',
 'angrezi',
 'par',
 'yeh',
 'tasweer']

In [99]:
f_zip = list(zip(vocab_roman, vocab_urdu))

In [100]:
print(f_zip)

[('wakeel', 'وکیل'), ('aik', 'ایک'), ('aisi', 'ایسی'), ('shakhsiyat', 'شخصیت'), ('ko', 'کو'), ('kaha', 'کہا'), ('jata', 'جاتا'), ('hai', 'ہے'), ('ke', 'کہ'), ('jo', 'جو'), ('qanoon', 'قانون'), ('1', 'شخص'), ('shakhs', 'جسے'), ('jisay', 'دوسرے'), ('dosray', 'کی'), ('ki', 'جگہ'), ('jagah', 'کام'), ('kaam', 'بازار'), ('bazaar', 'ضلع'), ('zila', 'لاطینی'), ('lateeni', 'ترکمانستان'), ('turkmenistan', 'کا'), ('ka', 'کے'), ('azlaa', 'اضلاع'), ('wala', 'والا'), ('railway', 'ریلوے'), ('station', 'اسٹیشن'), ('mein', 'میں'), ('waqay', 'واقع'), ('Pakistan', 'پاکستان'), ('wikipedia', 'وکیپیڈیا'), ('angrezi', 'انگریزی'), ('par', 'پر'), ('yeh', 'یہ'), ('tasweer', 'تصویر'), ('ne', 'ميں'), ('hi', 'نے'), ('aplod', 'ہی'), ('khud', 'اپلوڈ'), ('kisi', 'خود'), ('bhi', 'کسی'), ('tarteeb', 'بھی'), ('nafiz', 'ترتیب'), ('nahi', 'نافذ'), ('karta', 'نہیں'), ('logo', 'کرتا'), ('liye', 'لوگو'), ('nastalik', 'لیے'), ('khtati', 'نستعلیق'), ('is', 'خطاطی'), ('satar', 'اس'), ('se', 'سطر'), ('aap', 'سے'), ('tasaveer', '

Create a dictionary that maps each word to an index

In [101]:
roman_word_to_ix = {word: i for i, word in enumerate(vocab_roman)}
urdu_word_to_ix = {word: i for i, word in enumerate(vocab_urdu)}

REVERSE_roman_word_to_ix =  {v: k for k, v in roman_word_to_ix.items()}
REVERSE_urdu_word_to_ix =  {v: k for k, v in urdu_word_to_ix.items()}



print(roman_word_to_ix)
print(urdu_word_to_ix)


{'wakeel': 0, 'aik': 1, 'aisi': 2, 'shakhsiyat': 3, 'ko': 4, 'kaha': 5, 'jata': 6, 'hai': 7, 'ke': 8, 'jo': 9, 'qanoon': 10, '1': 11, 'shakhs': 12, 'jisay': 13, 'dosray': 14, 'ki': 15, 'jagah': 16, 'kaam': 17, 'bazaar': 18, 'zila': 19, 'lateeni': 20, 'turkmenistan': 21, 'ka': 22, 'azlaa': 23, 'wala': 24, 'railway': 25, 'station': 26, 'mein': 27, 'waqay': 28, 'Pakistan': 29, 'wikipedia': 30, 'angrezi': 31, 'par': 32, 'yeh': 33, 'tasweer': 34, 'ne': 35, 'hi': 36, 'aplod': 37, 'khud': 38, 'kisi': 39, 'bhi': 40, 'tarteeb': 41, 'nafiz': 42, 'nahi': 43, 'karta': 44, 'logo': 45, 'liye': 46, 'nastalik': 47, 'khtati': 48, 'is': 49, 'satar': 50, 'se': 51, 'aap': 52, 'tasaveer': 53, 'saada': 54, 'tareeqay': 55, 'khush': 56, 'aamdeed': 57, 'baat': 58, 'wajah': 59, 'policy': 60, 'sirf': 61, 'hazaar': 62, 'mazmoon': 63, 'hain': 64, 'jin': 65, 'asal': 66, 'har': 67, 'Sarif': 68, 'abna': 69, 'safha': 70, 'hota': 71, 'adad': 72, 'o': 73, 'shumaar': 74, 'sarfeen': 75, 'dayen': 76, 'janib': 77, 'talaash'

Convert To indeces

In [102]:
urdu_word_indices = torch.tensor([urdu_word_to_ix[word] for word in vocab_urdu], dtype=torch.long)
roman_word_indices = torch.tensor([roman_word_to_ix[word] for word in vocab_roman], dtype=torch.long)

In [103]:
print(urdu_word_indices)

tensor([   0,    1,    2,  ..., 5220, 5221, 5222])


In [104]:
print(roman_word_indices)


tensor([   0,    1,    2,  ..., 5137, 5138, 5139])


In [105]:


embedding = nn.Embedding(num_embeddings=10000, embedding_dim=128)


roman_embedded_words = embedding(roman_word_indices)
urdu_embedded_words = embedding(urdu_word_indices)

In [106]:
print(urdu_embedded_words)

tensor([[ 0.6776, -0.6124, -0.8693,  ...,  1.2455, -0.1795,  0.2714],
        [-0.0493,  1.9305,  1.2523,  ...,  0.2827, -0.8554,  0.8123],
        [-0.1102, -0.2845, -0.3517,  ..., -1.3665,  0.7974, -0.2982],
        ...,
        [ 0.7774,  2.0981,  0.2154,  ...,  0.4053,  0.3480,  0.0504],
        [ 0.8917, -0.1707,  0.1360,  ...,  0.6674,  1.9403,  1.0114],
        [-1.6150,  0.0703,  0.9880,  ...,  0.6246, -0.7504, -1.4652]],
       grad_fn=<EmbeddingBackward0>)


In [107]:
print(roman_embedded_words)

tensor([[ 0.6776, -0.6124, -0.8693,  ...,  1.2455, -0.1795,  0.2714],
        [-0.0493,  1.9305,  1.2523,  ...,  0.2827, -0.8554,  0.8123],
        [-0.1102, -0.2845, -0.3517,  ..., -1.3665,  0.7974, -0.2982],
        ...,
        [ 0.6526,  0.5862,  1.7351,  ...,  0.0607, -0.4422, -1.2831],
        [-0.1719,  0.0878,  1.7020,  ..., -0.0271, -0.7460,  1.7051],
        [-0.9854, -0.5193, -1.9010,  ..., -1.0299, -0.0178,  0.2586]],
       grad_fn=<EmbeddingBackward0>)


In [108]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Now we write the encorder Rnn class

In [109]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
    # loop 10 times to implement 10 cells of rnn
        for i in range(10):
            output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)





In [110]:
class Attention(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(Attention, self).__init__()
        self.dropout = torch.nn.Dropout(dropout)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.scale = torch.sqrt(torch.tensor(d_model, dtype=torch.float32))

    def forward(self, q, k, v, mask=None):
        # q, k, v have shape (batch_size, seq_len, d_model)
        scores = torch.matmul(q, k.transpose(-1, -2)) / self.scale

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attention_weights = self.softmax(scores)
        attention_weights = self.dropout(attention_weights)

        # Compute weighted sum of values
        output = torch.matmul(attention_weights, v)

        return output, attention_weights

Below is our Decoder Class

In [111]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers = 10):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size*2, hidden_size)
        self.attention = Attention(hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encorder_output):
       
        output = self.embedding(input).view(1, 1, self.hidden_size)
        context, attention_weights = self.attention(hidden[-1].unsqueeze(0), encorder_output, encorder_output)
        
        output = torch.cat((output[:, :, :self.hidden_size], context), dim=2)
        output, hidden = self.gru(output, hidden)
        # output = torch.cat((output, context), dim=2)

        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [112]:
# class DecoderRNN(nn.Module):
#     def __init__(self, output_size, hidden_size, num_layers = 10):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.attention = Attention
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden, encorder_output):
#         output = self.embedding(input).view(1, 1, -1)
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
    
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [113]:
roman_urdu_lines[2]

'wakeel aik aisi shakhsiyat ko kaha jata hai ke jo'

Now we make pairs for training

In [114]:
pairs = list(zip(roman_urdu_lines, urdu_lines))

In [115]:
pairs[10]

('wikipedia khud kisi bhi tarteeb ko nafiz nahi karta',
 'وکیپیڈیا خود کسی بھی ترتیب کو نافذ نہیں کرتا')

In [126]:
def index_from_sentence(idx, sentence):
    return [idx[word] for word in sentence.split(' ')]

def index_to_word(idx, index):
    return []

def tensor_from_sentence(idx, sentence):
    indexes = index_from_sentence(idx, sentence)
    # Can add EOS Token Here if needed
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(roman_word_to_ix, pair[0])
    target_tensor = tensor_from_sentence(urdu_word_to_ix, pair[1])
    return (input_tensor, target_tensor)


We need to convert a sentence into vectors

In [127]:
print(index_from_sentence(roman_word_to_ix, roman_urdu_lines[5]))

print(tensors_from_pair([roman_urdu_lines[23], urdu_lines[23]]))

[0, 18, 19, 20, 21, 22, 1, 21, 8, 23]
(tensor([[91],
        [ 8],
        [92],
        [93],
        [22],
        [94],
        [40],
        [95],
        [96],
        [97]]), tensor([[90],
        [22],
        [91],
        [92],
        [21],
        [93],
        [41],
        [94],
        [95],
        [96]]))


Now we write out training function. Below code is implementing a training loop for a sequence-to-sequence

In [128]:
# Define teach forcing ratio and max length
teacher_forcing_ratio = 0.5
MAX_LENGTH = 10

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = decoder_input = input_tensor[-1].unsqueeze(0) 
    
    decoder_hidden = encoder_hidden

  

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if di + 1 == target_length:
                break


    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Helper Functions

In [129]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [130]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [131]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [132]:
PAD_token = 3

In [133]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(roman_word_to_ix, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = input_tensor[0].view(1, 1)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if di == max_length - 1 or topi.item() == PAD_token:
                break
            else:
                decoded_words.append(REVERSE_urdu_word_to_ix[topi.item()])
            decoder_input = topi.squeeze().detach()
        
        return decoded_words, decoder_attentions[:di + 1]

In [134]:
hidden_size = 256
encoder1 = EncoderRNN(len(vocab_roman), hidden_size,).to(device)
attn_decoder1 = DecoderRNN( len(vocab_urdu), hidden_size).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=1000)

3m 35s (- 32m 17s) (1000 10%) 6.1992
6m 29s (- 25m 59s) (2000 20%) 5.4775
9m 34s (- 22m 19s) (3000 30%) 5.0311
12m 36s (- 18m 55s) (4000 40%) 4.7095
15m 27s (- 15m 27s) (5000 50%) 4.6152
18m 59s (- 12m 39s) (6000 60%) 4.4630
22m 12s (- 9m 31s) (7000 70%) 4.3406
25m 43s (- 6m 25s) (8000 80%) 4.0831
28m 55s (- 3m 12s) (9000 90%) 4.1910
32m 7s (- 0m 0s) (10000 100%) 3.9634


In [138]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Roman: ', pair[0])
        print('Urdu: ', pair[1])
        output_words,_ = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('predicted: ', output_sentence)
        print('')

In [139]:
evaluateRandomly(encoder1, attn_decoder1)

Roman:  woh January ko misar ke shehar iskandriya mein peda hue
Urdu:  وہ جنوری کو مصر کے شہر اسکندریہ میں پیدا ہوئے
predicted:  وہ اپنی کو کے پیدا کو پیدا کے پیدا

Roman:  Pakistan aam intikhabaat ke halqa pi pi mein umeedwaron ne
Urdu:  پاکستان عام انتخابات کے حلقہ پی پی میں امیدواروں نے
predicted:  پاکستان کے کی کے کے سے کے کے سے

Roman:  Pakistan riloyz ki sarkari Website ke mutabiq mughal railway station
Urdu:  پاکستان ریلویز کی سرکاری ویب سائٹ کے مطابق مغل ریلوے
predicted:  ریلویز کی سرکاری ویب سائٹ کے مطابق شاہ ریلوے

Roman:  Pakistan riloyz ki sarkari Website ke mutabiq paiir pyayi railway
Urdu:  پاکستان ریلویز کی سرکاری ویب سائٹ کے مطابق پیر پیائی
predicted:  ریلویز کی سرکاری ویب سائٹ کے مطابق شاہ آباد

Roman:  van town ship kaas county mshi gun angrezi ryast_haye mutahidda
Urdu:  وین ٹاؤن شپ کاس کاؤنٹی مشی گن انگریزی ریاستہائے متحدہ
predicted:  وہ تو میں تو میں وقت میں اس وقت

Roman:  pak tea house kayi nasheeb frazz se guzra aur kayi
Urdu:  پاک ٹی ہاؤس کئ نشیب فراز سے گزرا ا